In [1]:
addpath ("files")
setenv('GNUTERM','qt')

In [113]:
format long

In [128]:
function  [gfx] = gradiente(f, x, h = 1.e-07)

%{
    Aproximacion del gradiente por diferencias hacia adelante 
    de una funcion  de R^n a R.
    
    Input:
        f   .- apuntador a la función
        x   .- vector columna de dimension n.

    Output:

        gfx .- vector columna de dimension n, 
                 es la aproximacion al gradiente en x.
%}
    
    n = length(x);
    
    fx = f(x);                            
    xt = x;
    
    for i = 1:n
        xt(i) = xt(i) + h;
        fxh = f(xt);
        gfx(i) = (fxh - fx)/h;
        xt(i) = x(i);
    end
end

In [129]:
function [H] = hessian(f, x, h = 1.e-07)

    n = length(x);
    H = zeros(n);
    fx = f(x);

    for i = 1:n
        x1 = x;
        x1(i) = x1(i) + h;
        f1 = f(x1);
        for j = i:n
            x2 = x;
            x2(j) = x2(j) + h;
            f2 = f(x2);

            x3 = x1;
            x3(j) = x3(j) + h;
            f3 = f(x3);
            
            H(i,j) = (f3 - f2 - f1 + fx)/(h^2);
            H(j,i) = H(i,j);
        end
    end
end

In [150]:
function [W,x,iter] = metodoblhibrido(f, x, method='newton', tol=1.e-07, 
                                      maxiter = 300, maxjter = 20, c1 = 1.e-04)
%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Máximo descenso
        2. Método de Newton
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    iter = 0;        
    p = 1; 

    g  = gradiente(f,x)';
    W  = [x]; 

    while ( norm(g) > tol && iter < maxiter)

        if (strcmp(method,'max'))
            p = -g;
        end

        if (strcmp(method,'newton'))
            H = hessian(f,x);
            p = -H\g; 
        end

        %-----------------------------------------
        %            Búsqueda híbrida
        
        alfa = 1; jter = 0;   
        
        f_x  = f(x);
        xt   = x + alfa*p;        
        f1   = f(xt);
                  
        s  = p'*g;  
        if (abs(s) < 1.e-06)
            disp('No existe suficiente descenso  ')
            disp(sprintf('%2.0f  %2.10f',iter, s))
            break;
        end

        while(true)  % búsqueda de línea
        
            %-----------------------------------------
            %           Paso Backtracking
            alfa_back = alfa/2;
            f_back  = f(x + alfa_back*p);
            wolfe_back = false;
            
            %-----------------------------------------
            %           Paso Interpolación
            d2 = f1 - f_x - s;
            alfa_inter = -(s)/(2*d2); 
            f_inter =f(x + alfa_inter*p);
            wolfe_inter = false;
            
            %-----------------------------------------
            %    Paso backtracking cumple W1
            if (f_back < f_x + alfa*c1*s)
                wolfe_back = true;
            end
            
            %-----------------------------------------
            %    Paso interpolación cumple W1
            if (f_inter < f_x+alfa*c1*s)
                wolfe_inter = true;
            end
            
            if (wolfe_back && wolfe_inter)
                alfa = max(alfa_back,alfa_inter);
            elseif(wolfe_back)
                alfa = alfa_back;
            elseif(wolfe_inter)
                alfa = alfa_inter;
            else
                alfa = alfa_back;
            end
            
            if (wolfe_back || wolfe_inter)
                break;
            end
            
            if ((norm(alfa*p) < 1.e-3) || (jter > maxjter) )
                alfa = 1.e-2;
                break;
            end
            
            jter = jter +1;
        end   
        
        %          Fin de búsqueda híbrida
        %------------------------------------------
          
        W = [W, x];
        x = x + alfa*p;  
        fx = f(x); 
        g = gradiente(f,x)';
        iter = iter + 1;
        
    end      
end

In [151]:
function [ans] = shubert(X)
    if (length(size(X)) > 2)
        x = X(:,:,1);
        y = X(:,:,2);
    else
        x = X(1);
        y = X(2);
    end

    n = length(x);
    sum1 = zeros(1,n);
    sum2 = zeros(1,n);
    
    for i = 1:5
        sum1 = sum1 + i .* cos((i+1).*x+i);
        sum2 = sum2 + i .* cos((i+1).*y+i);
    end

    ans = sum1 .* sum2;
end

In [152]:
function [fx] = fcuad(x)
%{
    Función cuadrática f: Rn --> R

    Input:
        x  .- vector de longitud n
    Output:
        fx .- número real
%}
    fx = norm(x)^2 +1;
end

In [153]:
function [fx] = rosenbrock(x)
%{
    Función de Rosenbrock: f: R^2 --> R cuyo mínimo local es muy 
    difícil de alcanzar por medio de los métodos de optimización.
    
    Input:
         x.- vector de longitud 2
    Output:
        fx.- número real con el valor de la función.
%}
    fx = 100*(x(2) - x(1)^2)^2 + (1 - x(1))^2;
end

In [156]:
[W,x,iter] = metodoblhibrido(@shubert,[1,1]','newton',false);
x
fcuad(x)

[W,x,iter] = metodoblhibrido(@fcuad,[2,2,2,2]','newton',false);
x
fcuad(x)

[W,x,iter] = metodoblhibrido(@rosenbrock,[1.5,1.5]','newton',false);
x
rosenbrock(x)

No existe suficiente descenso  
 2  -0.0000000091
x =

   1.086518282703413
   1.086518282703413

ans =  3.361043957297547
No existe suficiente descenso  
 1  -0.0000000000
x =

  -2.400797143309319e-07
  -2.400797143309319e-07
  -2.400797143309319e-07
  -2.400797143309319e-07

ans =  1.000000000000230
No existe suficiente descenso  
11  -0.0000000361
x =

   1.000033237506457
   1.000054572786375

ans =    1.527366137204568e-08
